# Introduction:
For our final project we looked through different datasets that lead us to finding Squirrel Hill South as the best neighborhood in Pittsburgh. We did this through analyzing data from three different datasets and then combining this data to fit our metric. In order to come up with our metric, each member of our group found a dataset that interested them, and from these datasets we came up with a metric that fit all three. As an alternative, we could have came up with our metric and then explored datasets that fit this metric. Also, we have //not completed


# The Metric:
Our metric explores which neighborhood has is the best in terms of a well-rounded life. To determine this, we use datasets that show the median death age for each neighborhood, information on playgrounds, and data on street art in each neighborhood. For the playground data set, we took into account the number of playgrounds in each neighborhood that were ADA accessible. 

# The Best Neighborhood:
## Squirrel Hill South

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

play = pd.read_csv("https://data.wprdc.org/datastore/dump/e39ef76e-0a11-47c8-a86f-a37f55db7a2b")
public_art_data = pd.read_csv("public-art.csv", index_col="id")

# Conclusions:
## Hannah: 
Honestly, I do not have any experience with Pittsburgh neighborhoods. I've been living on campus, so I only know a very small amount of information about Oakland. I hadn't even heard of most of the neighborhoods I was gathering data on, but I have been through Squirrel Hill South on a bus before. It looks nice enough, and as far as our data shows, it proves to be the best neighborhoods in Pittsburgh. 